## UPC에 관하여
- 1자리: UPC 코드 첫째자리의 넘버시스템 캐릭터는 뒤의 나머지 숫자의 분류뿐만 아니라 의미를 해결하는데 도움을 준다.
    - 0, 6, 7, 8 : 아래를 제외한 모든 제품에 부여
    - 2 : 중량단위로 판매되는 가변 중량상품(random weight item: 육류,치즈 등)에 부여
    - 3 : 의약품, 건강관련제품에 부여
    - 4 : 점포내 판매를 목적으로 소매업자에 의해 마킹되는 제품에 부여(인스토아마킹용)
    - 5 : 쿠폰에 부여
    
- 5자리: 제조업체코드(Manufacturer Identification Number : 5 자리)
    - 제조업체코드는 UPC 코드관리기관인 UCC(Uniform Code Council)에서 각 제조업체에 부여합니다.
    
- 5자리: 상품품목코드(Item Code Number : 5자리 )
    - 상품품목코드는 제조업체가 부여하고 관리하는 코드입니다.
    
- 1자리: 체크디지트 (Check Digit : 1자리 )
    - EAN코드와 마찬가지로 모듈러스 10(modulus 10)방식으로 계산하여 부여하는 코드입니다.
    
----------------------------------------------------------------------
- `1유형` : 제조업체코드 5자리 중 뒤의 3자리가 000, 100, 200으로 끝나는 경우로, 단축형으로 사용할 수 있는 상품품목코드는 00000 ~ 00999로서 최대 1,000품목까지 사용이 가능합니다.

- `2유형` : 제조업체코드 5자리 중 뒤의 3자리가 300, 400, 500, 600, 700, 800, 900으로 끝나는 경우로, 단축형으로 사용할 수 있는 상품품목코드는 00000 ~ 00099로서 최대 100품목까지 사용이 가능합니다. 

- `3유형` : 제조업체코드 5자리 중 뒤의 2자리가 10, 20, 30, 40, 50, 60, 70, 80, 90으로 끝나는 경우로, 단축형으로 사용할 수 있는 상품품목코드는 00000 ~ 00009로서 최대 10품목까지 사용이 가능합니다. 

- `4유형` : 제조업체코드 5자리가 0으로 끝나지 않는 경우로 단축형으로 사용할 수 있는 상품품목 코드는 00005 ~ 00009로서 최대 5품목까지 사용이 가능합니다.

- 예를들어:
    - 1유형의경우... 

In [1]:
data = pd.read_csv('../data/train.csv')
data.head()

,TripType,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber
0,999,5,Friday,6.811315e+10,-1,FINANCIAL SERVICES,1000.0
1,30,7,Friday,6.053882e+10,1,SHOES,8931.0
2,30,7,Friday,7.410811e+09,1,PERSONAL CARE,4504.0
3,26,8,Friday,2.238404e+09,2,PAINT AND ACCESSORIES,3565.0
4,26,8,Friday,2.006614e+09,2,PAINT AND ACCESSORIES,1017.0


In [2]:
data['Upc'] = data['Upc'].fillna(value = 0)
data['Upc'] = data['Upc'].astype(int)
data.head()

,TripType,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber
0,999,5,Friday,68113152929,-1,FINANCIAL SERVICES,1000.0
1,30,7,Friday,60538815980,1,SHOES,8931.0
2,30,7,Friday,7410811099,1,PERSONAL CARE,4504.0
3,26,8,Friday,2238403510,2,PAINT AND ACCESSORIES,3565.0
4,26,8,Friday,2006613744,2,PAINT AND ACCESSORIES,1017.0


In [3]:
def check_len(upc):
    return len(str(upc))

### upc 바코드 길이 체크

In [4]:
data['len(upc)'] = data['Upc'].apply(check_len)

In [5]:
s = list(data['len(upc)'].unique())
s.sort()
for idx in s:

    count = len(data[data['len(upc)'].isin([idx])])
    print(idx," :  ",count)

# 1은 missing data

1  :   4129
3  :   7
4  :   29745
5  :   372
7  :   1
8  :   412
9  :   2166
10  :   433341
11  :   168418
12  :   8463


### upc 바코드 전처리
- 1.현재 주어진 upc바코드는 끝자리 check digit 숫자가 존재하지 않는다. 
https://www.gs1.org/services/how-calculate-check-digit-manually
    - 먼저 check digit을 구한다.

- 2.길이를 12자리, 13자리로 맞추자.
https://www.cognex.com/ko-kr/resources/symbologies/1-d-linear-barcodes/ean-13-barcodes
    - UPC-A, EAN-13과 동일한 미국 바코드. 13자리 바코드는 처음 1자리수가 country code를 뜻하고 그 외에는 UPC규칙과 동일
    
- 3.회사와 제품을 구분하는 컬럼으로 새로 만들기

#### 1. check digit 숫자를 구하기

In [6]:
# check digit 구하는 함수

def get_checkdigit(upc):
    
    odd_s = [int(i) for i in str(upc)[-1::-2]]
    even_s = [int(i) for i in str(upc)[-2::-2]]
    
    odd_sum = sum(list(map(lambda number : number * 3, odd_s)))
    even_sum = sum(list(map(lambda number : number, even_s)))
    check_sum = (odd_sum + even_sum) % 10
    

    if check_sum == 0:
        return check_sum
    else:
        return 10 - check_sum

#### 2. 길이를 12자리, 13자리로 맞추기
- 앞자리는 0으로 채움

In [7]:
def make_full_upc(upc):
    if upc == 0:
        return "missing"
    else:
        tmp_upc = str(upc) + str(get_checkdigit(upc))


        if len(tmp_upc) < 12:
            length = 12 - len(tmp_upc)
            zero = "0" * length
            standard_upc = zero + tmp_upc
            return standard_upc
        else:
            return tmp_upc 

In [8]:
# test

print(get_checkdigit(2238403510))
print(make_full_upc(2238403510))

2
022384035102


In [9]:
data['standard_upc'] = data['Upc'].apply(make_full_upc)

In [10]:
data['standard_upc(len)'] = data['standard_upc'].apply(check_len)

In [11]:
# missing 제외 12, 13자리로 변환
data['standard_upc(len)'].unique()

array([12,  7, 13])

#### 3. 컬럼추가

In [12]:
def classify_company_product(standard_upc):
    class_ = []
    
    if standard_upc == "missing":
        return standard_upc
    
    elif len(standard_upc) == 13:
        company = standard_upc[1:7]
        product = standard_upc[7:13]
        
        class_.append(company)
        class_.append(product)
        return class_
    
    # len = 12
    else: 
        company = standard_upc[:6]
        product = standard_upc[6:12]
        
        class_.append(company)
        class_.append(product)
        return class_

In [13]:
data['tmp'] = data['standard_upc'].apply(classify_company_product)

In [14]:
data.head()

,TripType,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber,len(upc),standard_upc,standard_upc(len),tmp
0,999,5,Friday,68113152929,-1,FINANCIAL SERVICES,1000.0,11,681131529297,12,"[681131, 529297]"
1,30,7,Friday,60538815980,1,SHOES,8931.0,11,605388159809,12,"[605388, 159809]"
2,30,7,Friday,7410811099,1,PERSONAL CARE,4504.0,10,074108110992,12,"[074108, 110992]"
3,26,8,Friday,2238403510,2,PAINT AND ACCESSORIES,3565.0,10,022384035102,12,"[022384, 035102]"
4,26,8,Friday,2006613744,2,PAINT AND ACCESSORIES,1017.0,10,020066137441,12,"[020066, 137441]"


In [15]:
def make_company_col(tmp):
    return tmp[0]

def make_product_col(tmp):
    return tmp[1]

data['company_code'] = data['tmp'].apply(make_company_col)
data['product_code'] = data['tmp'].apply(make_product_col)

In [16]:
data.head()

,TripType,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber,len(upc),standard_upc,standard_upc(len),tmp,company_code,product_code
0,999,5,Friday,68113152929,-1,FINANCIAL SERVICES,1000.0,11,681131529297,12,"[681131, 529297]",681131,529297
1,30,7,Friday,60538815980,1,SHOES,8931.0,11,605388159809,12,"[605388, 159809]",605388,159809
2,30,7,Friday,7410811099,1,PERSONAL CARE,4504.0,10,074108110992,12,"[074108, 110992]",074108,110992
3,26,8,Friday,2238403510,2,PAINT AND ACCESSORIES,3565.0,10,022384035102,12,"[022384, 035102]",022384,035102
4,26,8,Friday,2006613744,2,PAINT AND ACCESSORIES,1017.0,10,020066137441,12,"[020066, 137441]",020066,137441


#### 불필요 컬럼 제거

In [17]:
del data['Upc'], data['len(upc)'], data['tmp'], data['standard_upc(len)']

In [18]:
data.head()

,TripType,VisitNumber,Weekday,ScanCount,DepartmentDescription,FinelineNumber,standard_upc,company_code,product_code
0,999,5,Friday,-1,FINANCIAL SERVICES,1000.0,681131529297,681131,529297
1,30,7,Friday,1,SHOES,8931.0,605388159809,605388,159809
2,30,7,Friday,1,PERSONAL CARE,4504.0,074108110992,074108,110992
3,26,8,Friday,2,PAINT AND ACCESSORIES,3565.0,022384035102,022384,035102
4,26,8,Friday,2,PAINT AND ACCESSORIES,1017.0,020066137441,020066,137441


In [20]:
len(data)

647054

In [22]:
len(data['company_code'].unique())

5893

In [23]:
len(data['product_code'].unique())

88135

### 결과:

- 총 647054 데이터 중 유니크한 company code 5893으로 분류
- 